# Astronomical Source Detection: SEP Tutorial

Eric Ryan authored this Jupyter notebook

First, we will import required modules and packages, and prepare our notebook to read the image and display the plots.

In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt

from matplotlib import rcParams
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

%matplotlib inline 

rcParams['figure.figsize'] = [10.,8.]

Next, we will open and display the desired image from a FITS file 

In [ ]:
image_data = get_pkg_data_filename("/Users/ericryan/Downloads/image.fits")
data = fits.getdata(image_data, ext=0)

m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig("/Users/ericryan/Downloads/data.png")

## Background subtraction

Now, we will subtract the optical/IR data from the background in order to detect the sources.

The first step to eliminating background data is to estimate the spatially varying background and spatially varying background noise level.

In [ ]:
bkg = sep.Background(data)

With this object, we can get a "global" mean and noise of the the image background.

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Next, we evaluate the background as 2-dimensional array of the same size as the original image.

In [ ]:
bkg_image = bkg.back()

Now we can show the background image and save the figure to a PNG file.

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("/Users/ericryan/Downloads/bkg_image.png")

Similarly to above, we will evaluate the background noise as 2-dimensional array of the same size as the original image.

In [ ]:
bkg_rms = bkg.rms()

Now we can show the background noise and save the figure to a PNG file.

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("/Users/ericryan/Downloads/bkg_rms.png")

Finally, we will subtract the background from the original image.

In [ ]:
data_sub = data - bkg

## Object detection

Now that we have subtracted the background from the original image, we can run object detection on the background-subtracted data.

We will set the detection threshold to be a contant value of 1.5σ, where σ is the global background RMS.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

We can print the number of sources found in the image.

In [ ]:
len(objects)

Now comes the fun part of plotting an ellipise for each object found in the background-subtracted image.

First we plot the background-subtracted image, then using the centroid coordinates of the objects we will plot a red ellipse using some simple width, height, and angle parameters.

Finally, we will save the figure to a PNG file.

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i], angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig("/Users/ericryan/Downloads/detected_objects.png")

Here, we will list the available fields of 'objects'.

In [ ]:
objects.dtype.names

## Aperture Photometry

Last, but not least, we will perform a circular aperture photometry with a 3 pixel radius at the locations of the object in the background-subtracted image.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

Now we will show the results of the first 10 objects.

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))